In [1]:
import re
import pandas as pd
from os import walk
import numpy as np
import pickle

In [2]:
#https://stackoverflow.com/questions/2319019/using-regex-to-remove-comments-from-source-files
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        # if the 2nd group (capturing comments) is not None,
        # it means we have captured a non-quoted (real) comment string.
        if match.group(2) is not None:
            return "" # so we will return empty to remove the comment
        else: # otherwise, we will return the 1st group
            return match.group(1) # captured quoted-string
    return regex.sub(_replacer, string)

#following the example above, distinguish group(1): "" or'' and group(2,true comments): \*.*\ or //
#return when a true comment is located
def leave_only_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    comments = [x[1] for x in regex.findall(script)]
    return comments

In [3]:
#read where the sol source code files are
mypath = 'sol_source/'

sol_files = []
for (dirpath, dirnames, filenames) in walk(mypath):
    sol_files.extend(filenames)
    break

In [4]:
#read every sol source file and record their name, address, and content.
#also separate codes and comments
name_list = []
address_list = []
uncommented = []
source_code = []
comments = []

for file in sol_files:
    #print(file)
    address = file.split('_')[0]
    address_list.append(address)
    name = file.split('_')[1].split('.')[0]
    name_list.append(name)
    
    path = 'sol_source/' + file
    #f = open(path, "rb")
    f = open(path, encoding="utf-8")    
    script = f.read()
    f.close()
    
    source_code.append(script)
    uncommented.append(remove_comments(script))
    comments.append(leave_only_comments(script))

In [5]:
#form dataframe
table = [name_list,address_list,source_code,uncommented,comments]
table = pd.DataFrame(table).transpose()
table.columns = ['name','address','souce_code','uncommented','comments']

In [6]:
#load and merge slug name from earlier file
slug = pd.read_csv('HU_Projects-master/DApps.csv')
table = table.merge(slug.loc[:,('address','slug')],on='address',how='left')

In [7]:
#save to pickle
table.to_pickle('my_df.pickle')